# LLMSの応用(1)：Langchainの基礎

[LangChain](https://www.langchain.com/)は大規模言語モデルの機能拡張を効率的に実装するためのライブラリです。

- ```llms```: 言語モデルを呼び出すためのラッパーを提供します
- ```prompts```: プロンプトのテンプレートを作成する機能を提供します
- ```chains```: ひとつのワークフロー内で LLM やプロンプトテンプレートを組み合わせて使用するための機能を提供します
- ```agents```: エージェントを使用することで、課題の解決順序をも LLM を用いて決定し、実行させることができます
- ```memory```: チェーンとエージェントに状態を持たせるための機能を提供します

In [1]:
!pip install langchain==0.1.12
!pip install langchain-community
!pip install langchain-core
!pip install langsmith
!pip install openai
!pip install python-dotenv
!pip install langchain-openai
!pip install -U --quiet langchain-google-genai pillow
!pip install google-generativeai
!pip install google-search-results numexpr wikipedia langchain-experimental
!pip install duckduckgo-search

  Using cached langchain_community-0.0.38-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_text_splitters-0.0.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl.metadata (14 kB)
Using cached langchain_community-0.0.38-py3-none-any.whl (2.0 MB)
Using cached langchain_core-0.1.53-py3-none-any.whl (303 kB)
Using cached langchain_text_splitters-0.0.2-py3-none-any.whl (23 kB)
Using cached langsmith-0.1.147-py3-none-any.whl (311 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.6.4
    Uninstalling langsmith-0.6.4:
      Successfully uninstalled langsmith-0.6.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.7
    Uninstalling langchain-core-1.2.7:
      Successfully uninstalled langchain-core-1.2.7
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-split

## OpenAI/Gemini API

[OpenAI](https://openai.com/blog/openai-api)と[Google(Gemini)](https://deepmind.google/technologies/gemini/#introduction)が提供するAPIを通じて、多岐にわたるAIモデルへのアクセスを可能になります。

APIを使うためには、まず「自分がサービスを利用できる証」となる「APIキー」を発行する必要があります。そして、OpenAIのAPIは、このAPIキーによって利用した使用量に応じて、課金される仕組みです。

そのため、APIキーが外部に漏れると、他者によって不正に使用されて料金が発生してしまうため、他の人へ共有しないように注意しましょう。

LangChainは、さまざまな LLM に汎用インターフェースを提供し、ユーザーがAPIを介してさまざまなモデルを操作できるようにします。

In [2]:
from dotenv import load_dotenv
import os
import getpass
from langchain_openai import OpenAI, ChatOpenAI     

load_dotenv(dotenv_path='../.env')
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

/home/lvzeyu/anaconda3/envs/jupyterbook/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### OpenAI API

`OpenAI`クラスのインスタンスでGPT-3モデルを使用するための設定を行っています。

- `model_name`という引数で、使用する[モデルの名前](https://platform.openai.com/docs/models)を指定します。


- `temperature`という引数は、生成されるテキストのランダム性を制御します。`temperature`が高いほど（1に近いほど）、出力はよりランダムになります。逆に、`temperature`が低いほど（0に近いほど）、モデルの出力はより一貫性があり、予測可能になります。

- `max_tokens`という引数は、生成するテキストの最大トークン数を指定します。この場合、生成されるテキストは最大で256トークンになります。トークンとは、テキストを分割した単位のことで、一般的には単語や句読点などが1トークンとなります。


In [5]:
llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0.9,
    max_tokens=512
)

In [6]:
llm.invoke("東北大学を紹介してください：")

'\n\n東北大学は、東北地方の主要な大学であり、日本を代表する国立総合大学の一つです。国内外から多くの優秀な学生を集め、幅広い学問分野で高いレベルの教育・研究活動を行っています。その歴史は1897年に創立された東北帝国大学に始まり、戦前からの伝統と実績を持ちます。\n\n東北大学には5つのキャンパスがあり、宮城キャンパスは国際的なキャンパスとして知られています。日本語教育センターや留学生センターが設置されており、留学生向けのサポート体制が整っています。また、海外の大学との交流プログラムも多数あり、国際的な視野を持った学生を育成することに力を入れています。\n\n東北大学は、全国的に高い評価を受ける研究機関としても知られています。特に科学技術分野では、国内外から多くの注目を集めており、ノーベル賞受賞者も輩出しています。また、他大学や企業との産学連携も積極的に行われており、社会に貢献する研究開発活動を行っています。\n\n東北大学には、学部・大学院を含めて約10万人の学生が在籍しており、多様な文化やバックグラウンドを持つ学生たちが集う国際的な大学です。学生生活を充実させるためのサ'

### Gemini API

#### Gemini APIの設定


In [7]:
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [8]:
import google.generativeai as genai
models = [m for m in genai.list_models()]
models

/home/lvzeyu/anaconda3/envs/jupyterbook/lib/python3.10/site-packages/google/api_core/_python_version_support.py:275: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/tmp/ipykernel_151443/3039597036.py:1: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


[Model(name='models/embedding-gecko-001',
       base_model_id='',
       version='001',
       display_name='Embedding Gecko',
       description='Obtain a distributed representation of a text.',
       input_token_limit=1024,
       output_token_limit=1,
       supported_generation_methods=['embedText', 'countTextTokens'],
       temperature=None,
       max_temperature=None,
       top_p=None,
       top_k=None),
 Model(name='models/gemini-2.5-flash',
       base_model_id='',
       version='001',
       display_name='Gemini 2.5 Flash',
       description=('Stable version of Gemini 2.5 Flash, our mid-size multimodal model that '
                    'supports up to 1 million tokens, released in June of 2025.'),
       input_token_limit=1048576,
       output_token_limit=65536,
       supported_generation_methods=['generateContent',
                                     'countTokens',
                                     'createCachedContent',
                                     'batc

In [9]:
import google.generativeai as genai

llm = genai.GenerativeModel('models/gemini-2.5-flash')

In [10]:
response = llm.generate_content("東北大学を紹介してください：")

In [11]:
from IPython.display import Markdown
Markdown(response.text)

東北大学は、宮城県仙台市に本部を置く、日本を代表する国立大学の一つです。その特徴と魅力をいくつかご紹介します。

### 東北大学の主な特徴

1.  **旧帝国大学としての歴史と伝統**
    *   1907年に創立された旧帝国大学（旧帝大）の一つであり、国内では京都大学、東京大学に次ぐ3番目の帝国大学として設立されました。
    *   「研究第一主義」「門戸開放」「実学尊重」を建学以来の理念として掲げています。

2.  **「門戸開放」の精神**
    *   特に注目すべきは、創立当初から「門戸開放」の理念を強く打ち出していたことです。
    *   1913年には、**日本の大学として初めて女子学生の入学を許可**し、また専門学校卒業生にも広く門戸を開くなど、当時としては非常に先進的な取り組みを行いました。

3.  **「研究第一主義」と高い研究力**
    *   建学以来、「研究第一主義」を掲げ、基礎研究から応用研究まで幅広い分野で世界をリードする研究成果を生み出しています。
    *   特に**材料科学、半導体、スピントロニクス、災害科学、情報科学、医学**などの分野で国際的に高い評価を受けています。
    *   ノーベル賞受賞者や文化勲章受章者など、多くの著名な研究者を輩出しています。

4.  **国際的な評価とランキング**
    *   国内外の大学ランキングでも常に上位に位置し、世界トップクラスの教育・研究機関として認知されています。
    *   特に、Times Higher Education (THE) の「世界大学ランキング」では、日本の大学として上位常連校です。

5.  **緑豊かなキャンパスと立地**
    *   「杜の都」と呼ばれる緑豊かな仙台市に位置しており、青葉山、川内、星陵といった広大なキャンパスを有しています。
    *   学術都市としての仙台は、落ち着いた環境でありながら、都市機能も充実しており、学生生活を送るには非常に良い環境です。

6.  **社会貢献と地域連携**
    *   東日本大震災の際には、災害科学国際研究所を設立するなど、震災からの復興と防災・減災研究において中心的な役割を担ってきました。
    *   地域社会との連携も密接で、産学官連携を通じて地域の発展にも大きく貢献しています。

### 学部・研究科構成（主要なもの）

*   **学部:** 文学部、教育学部、法学部、経済学部、理学部、医学部、歯学部、薬学部、工学部、農学部
*   **大学院:** 上記の各研究科に加え、生命科学研究科、環境科学研究科、情報科学研究科、医工学研究科、国際文化研究科、教育情報学研究部・教育部、高等司法研究科、災害科学国際研究所など、多岐にわたる専門分野の研究科を有しています。

### まとめ

東北大学は、その長い歴史の中で培われた「門戸開放」と「研究第一主義」の精神を基盤に、常に時代の最先端を走り続けている大学です。学術的な深さと広さを兼ね備え、国内外から優れた学生や研究者が集まる場として、未来を切り拓く人材の育成と、人類社会の発展に貢献し続けています。

## Prompt Templates


プロンプトテンプレートは、プロンプトを作成する再現可能な方法を指します。これには、エンドユーザーから一連のパラメーターを受け取り、プロンプトを生成するテキスト文字列(テンプレート)が含まれます。

In [12]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["language","text"],
    template="次の日本語のテキストを{language}に翻訳してください：{text}",
)


In [13]:
print(prompt.format(language="英語", text="東北大学は日本の東北地方にある大学です。"))

次の日本語のテキストを英語に翻訳してください：東北大学は日本の東北地方にある大学です。


## チェーン

### チェーンの基本

LLM は単独でも十分に強力に機能します。 しかし、 LLM 同士を組合わせたり、ある機能に特化した他のモジュールとともに利用することで、より複雑なアプリケーションを構築することができます。 LangChain では、このような他の機能と連結するための汎用的なインターフェースとして、チェーンを提供しています。 チェーンを用いることで、LLM の利用を含む "一連の処理" を一つのまとまりとして扱うことができます。

つまり、平易な言葉でいえば、チェーンは「複数の処理の連なり」です。 この処理の連鎖の部品となるチェーンの構成要素のことを リンク と呼びます。 リンクの一例は、LLM の呼び出しなどの基本的な処理です。さらには、その他のチェーン全体をリンクとして含むチェーンも作成できます。

チェーンの代表例は、LLM とプロンプトテンプレートを組合わせて使用するための```LLMChain```です。 このチェーンを用いると、

- ユーザーの入力を受け取り
- それをPromptTemplateでフォーマットし
- フォーマットされたレスポンスを LLM に渡す

という一連の操作を一つのまとまりとして実行できます。 

基本的な使い方としては、LLM や プロンプトテンプレートなどの基本要素を組み合わせて使用することが考えられます。

In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash", google_api_key=os.getenv("GOOGLE_API_KEY"))


In [15]:
chain = prompt | llm
chain.invoke(
    {"language":"英語", "text":"東北大学は日本の東北地方にある大学です。"}
)

AIMessage(content="Here's the English translation:\n\nTohoku University is a university in the Tohoku region of Japan.", additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bde1f-8a1b-7683-87a7-af501dfb2ace-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 21, 'output_tokens': 371, 'total_tokens': 392, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 349}})

In [16]:
chain.invoke({"language":"中国語", "text":"東北大学は日本の東北地方にある大学です。"})

AIMessage(content='东北大学是位于日本东北地区的一所大学。', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bde1f-9935-7232-a53f-48220bdfc506-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 22, 'output_tokens': 690, 'total_tokens': 712, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 679}})

### 実装例:Few Shot Learning

In [17]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate


In [18]:
examples = [
    {"word": "楽しい", "antonym": "悲しい"},
    {"word": "高い", "antonym": "低い"},
]

In [19]:
example_formatter_template = """
Word: {word}
Antonym: {antonym}\n
"""

example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template,
)


- `examples`: モデルに示す例を指定します。
- `example_prompt`: 例をどのように提示するかを指定します。
- `prefix`: プロンプトの前置詞を指定します。一般的には、モデルにタスクを説明するためのものです。
- `suffix`: プロンプトの後置詞を指定します。一般的には、モデルに入力と出力の形式を示すためのものです。
- `input_variables`: 入力の変数名を指定します。
- `example_separator`: 例を区切るための文字列を指定します。

In [20]:
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Word: {input}\nAntonym:",
    input_variables=["input"],
    example_separator="\n",
)

In [ ]:
print(few_shot_prompt.format(input="大きい"))

Give the antonym of every input

Word: 楽しい
Antonym: 悲しい



Word: 高い
Antonym: 低い


Word: 大きい
Antonym:


In [ ]:
llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash", google_api_key=os.getenv("GOOGLE_API_KEY"))


In [ ]:
from langchain_core.output_parsers import StrOutputParser

chain = few_shot_prompt | llm | StrOutputParser()

var = few_shot_prompt.input_variables[0] 
print(chain.invoke({var: "大きい"}))

小さい


#### 課題

Few Shot Learningでセンチメント分析を実装しなさい。

- データフレームから一部のテキストとラベル($n=5$)を抽出し、`examples`を作成します
- Few Shot Learningためのpromptを作成します
- chainを作成し、任意のテキストに対するセンチメント予測結果を確認します

In [ ]:
from datasets import load_dataset
dataset = load_dataset("imdb")

In [ ]:
df_train_sample = dataset["train"].to_pandas().sample(5, random_state = 123)

In [24]:
df_train_sample

,text,label
20000,After reading some quite negative views for th...,1
5515,Really no reason to examine this much further ...,0
966,"""Happy Go Lovely"" has only two things going fo...",0
22726,This movie is the first of Miikes triad societ...,1
2690,"Normally I would never rent a movie like this,...",0
